<a href="https://colab.research.google.com/github/joselopez11c/mi-primer-proyecto/blob/main/remates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install pdfplumber
!pip install curlify
!pip install openpyxl

import requests
import re
import os
import pandas as pd
import pdfplumber
from bs4 import BeautifulSoup
from google.colab import drive

In [53]:
url = 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml'
urlLogin = 'https://remaju.pj.gob.pe/remaju/pages/seguridad/login.xhtml'

headers = {
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,es;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Faces-Request': 'partial/ajax',
    'Origin': 'https://remaju.pj.gob.pe',
    'Referer': 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

response = requests.get(urlLogin, headers=headers)
cookie_header = 'JSESSIONID=' + response.cookies.get_dict().get('JSESSIONID')
headers['Cookie'] = cookie_header

def fetch_page(first_row):
  dataPagination = {
      'javax.faces.partial.ajax': 'true',
      'javax.faces.source': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.execute': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.render': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.behavior.event': 'page',
      'javax.faces.partial.event': 'page',
      'formBuscarRemateExterno:listaRemate_pagination': 'true',
      'formBuscarRemateExterno:listaRemate_skipChildren': 'true',
      'formBuscarRemateExterno:listaRemate_first': str(first_row),
      'formBuscarRemateExterno:listaRemate_rows': '12',
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
  }
  response = requests.post(url, headers=headers, data=dataPagination)
  return response

def downloadFile(idFile):
  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': cookie_header
  }

  dataFile = {
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
      idFile: '',
      'javax.faces.ViewState': view_state_value
  }

  response = requests.post(url, headers=headers, data=dataFile)
  return response


page = fetch_page(3) ####################################################### LLAMADA A LA PAGINA

soup = BeautifulSoup(page.content, 'xml')

update_tag = soup.find('update', {'id': 'j_id1:javax.faces.ViewState:0'})
view_state_value = update_tag.text

cdata_content = soup.find('update').text

# Ahora parseamos el contenido del CDATA, que es HTML
soup_html = BeautifulSoup(cdata_content, 'html.parser')

# Buscar todos los div con la clase 'card azul'
cards = soup_html.find_all('div', class_='card azul')

data = []
for card in cards:
    # Obtener número de remate y convocatoria
    remate_info = card.find('span', class_='text-bold label-danger h6').get_text(strip=True) if card.find('span', class_='text-bold label-danger h6') else 'No disponible'

    # Dividir la información de remate en número y convocatoria
    if remate_info != 'No disponible':
        parts = remate_info.split(" - ")
        remate_number = parts[0] if len(parts) > 0 else 'No disponible'
        convocatoria = parts[1] if len(parts) > 1 else 'No disponible'
    else:
        remate_number = 'No disponible'
        convocatoria = 'No disponible'

    # Ubicación, buscando la etiqueta específica y extrayendo el texto siguiente
    location_tag = card.find('i', class_='fa fa-map-marker')
    location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'

    # Descripción del inmueble, buscando el contenedor específico
    description = card.find('div', class_='texto-info-scroll').get_text(strip=True) if card.find('div', class_='texto-info-scroll') else 'No disponible'

    # Buscar todos los scripts dentro del card
    scripts = card.find_all('script')
    script_id = scripts[-1].get('id')[:-2] if scripts and scripts[-1].get('id').endswith('_s') else 'Script ID no encontrado'

    # Guardar los datos extraídos en la lista
    data.append({
        'Numer': remate_number,
       # 'Convocation': convocatoria,
       # 'Location': location,
       # 'Description': description,
        'Script ID': script_id,
       # 'View Code' : view_state_value,
       # 'Cookie header' : cookie_header
    })

# Crear un DataFrame con los datos recolectados
df = pd.DataFrame(data)

df['Demandante'] = ""
df['Demandado'] = ""
df['Expediente_Judicial'] = ""
df['Juzgado'] = ""

drive.mount('/content/drive')
save_folder = '/content/drive/My Drive/remates'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)


for index, row in df.iterrows():
    file_name = row['Numer'] + '.pdf'
    response = downloadFile(row['Script ID'])  # Asegúrate de que esta función esté correctamente definida

    if response.ok and 'application/pdf' in response.headers.get('Content-Type', ''):
        pdf_path = os.path.join(save_folder, file_name)
        with open(pdf_path, 'wb') as f:
            f.write(response.content)

        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
              text += page.extract_text().replace('\n', ' ') if page.extract_text() else ''

        # Usar expresiones regulares para extraer la información
        #print(file_name + ': ' + text)
        demandante = re.search(r"seguidos por (.*?),", text, re.DOTALL)
        demandado = re.search(r", contra (.*?), sobre", text, re.DOTALL)
        expediente = re.search(r"Expediente Judicial (.*?),", text, re.DOTALL)
        juzgado = re.search(r"tramitado ante el (.*?),", text, re.DOTALL)

        df.at[index, 'Demandante'] = demandante.group(1) if demandante else "No encontrado"
        df.at[index, 'Demandado'] = demandado.group(1) if demandado else "No encontrado"
        df.at[index, 'Expediente_Judicial'] = expediente.group(1) if expediente else "No encontrado"
        df.at[index, 'Juzgado'] = juzgado.group(1).replace('\n', ' ').strip() if juzgado else "No encontrado"
        # Solo eliminar el archivo si no es del Banco de Crédito del Perú
        if "BANCO DE CREDITO DEL PERU" not in demandante.group(1) if demandante else "":
            os.remove(pdf_path)

    else:
        print("Failed to download a valid PDF file.")

df = filtered_df = df[df['Demandante'].str.contains("BANCO DE CREDITO DEL PERU", na=False)]

columns_to_include = ['Numer', 'Demandante', 'Demandado', 'Expediente_Judicial', 'Juzgado']
df_to_save = df[columns_to_include]

excel_path = os.path.join(save_folder, 'Remates_Data.xlsx')

# Usando openpyxl como motor para escribir archivos .xlsx
df_to_save.to_excel(excel_path, index=False, engine='openpyxl')

df_to_save

<ipython-input-53-c0b80373946c>:89: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Numer,Demandante,Demandado,Expediente_Judicial,Juzgado
1,Remate N° 14062,BANCO DE CREDITO DEL PERU,JAMES ANTHONY CORTEZ SANTIAGO,N° 00782-2021-0-2001-JR-CI- 03,3° JUZG. CIVIL -Av.F.Chirichigno 351-Urb.El Ch...


In [5]:
# Definir constantes y configuraciones iniciales
url = 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml'
urlLogin = 'https://remaju.pj.gob.pe/remaju/pages/seguridad/login.xhtml'
save_folder = '/content/drive/My Drive/remates'

headers = {
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,es;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Faces-Request': 'partial/ajax',
    'Origin': 'https://remaju.pj.gob.pe',
    'Referer': 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

def login():
    response = requests.get(urlLogin, headers=headers)
    return 'JSESSIONID=' + response.cookies.get_dict().get('JSESSIONID')

headers['Cookie'] = login()

def fetch_page(first_row):
    dataPagination = {
        'javax.faces.partial.ajax': 'true',
        'javax.faces.source': 'formBuscarRemateExterno:listaRemate',
        'javax.faces.partial.execute': 'formBuscarRemateExterno:listaRemate',
        'javax.faces.partial.render': 'formBuscarRemateExterno:listaRemate',
        'javax.faces.behavior.event': 'page',
        'javax.faces.partial.event': 'page',
        'formBuscarRemateExterno:listaRemate_pagination': 'true',
        'formBuscarRemateExterno:listaRemate_skipChildren': 'true',
        'formBuscarRemateExterno:listaRemate_first': str(first_row),
        'formBuscarRemateExterno:listaRemate_rows': '12',
        'formBuscarRemateExterno': 'formBuscarRemateExterno',
    }
    return requests.post(url, headers=headers, data=dataPagination)

def parse_html(content):
    soup = BeautifulSoup(content, 'xml')
    view_state_value = soup.find('update', {'id': 'j_id1:javax.faces.ViewState:0'}).text
    cdata_content = soup.find('update').text
    return BeautifulSoup(cdata_content, 'html.parser'), view_state_value

def extract_data(soup_html):
    cards = soup_html.find_all('div', class_='card azul')
    data = []
    for card in cards:
        remate_info = card.find('span', class_='text-bold label-danger h6').get_text(strip=True) if card.find('span', class_='text-bold label-danger h6') else 'No disponible'
        if remate_info != 'No disponible':
            parts = remate_info.split(" - ")
            remate_number = parts[0] if len(parts) > 0 else 'No disponible'
            convocatoria = parts[1] if len(parts) > 1 else 'No disponible'
        else:
            remate_number = 'No disponible'
            convocatoria = 'No disponible'
        location_tag = card.find('i', class_='fa fa-map-marker')
        location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'
        description = card.find('div', class_='texto-info-scroll').get_text(strip=True) if card.find('div', class_='texto-info-scroll') else 'No disponible'
        scripts = card.find_all('script')
        script_id = scripts[-1].get('id')[:-2] if scripts and scripts[-1].get('id').endswith('_s') else 'Script ID no encontrado'
        data.append({
            'Número': remate_number,
            'Convocatoria': convocatoria,
            'Ubicación': location,
            'Descripción': description,
            'Script ID': script_id
        })
    return data

def downloadFile(script_id):
    headers = {
        'Accept': 'application/xml, text/xml, */*; q=0.01',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cookie': headers['Cookie']
    }
    dataFile = {
        'formBuscarRemateExterno': 'formBuscarRemateExterno',
        script_id: '',
        'javax.faces.ViewState': view_state_value
    }
    return requests.post(url, headers=headers, data=dataFile)

def process_pdf(response, index, df):
    if response.ok and 'application/pdf' in response.headers.get('Content-Type', ''):
        file_name = df.at[index, 'Número'] + '.pdf'
        pdf_path = os.path.join(save_folder, file_name)
        with open(pdf_path, 'wb') as f:
            f.write(response.content)
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text().replace('\n', ' ') if page.extract_text() else ''
        demandante = re.search(r"seguidos por (.*?),", text, re.DOTALL)
        demandado = re.search(r", contra (.*?), sobre", text, re.DOTALL)
        expediente = re.search(r"Expediente Judicial (.*?),", text, re.DOTALL)
        juzgado = re.search(r"tramitado ante el (.*?),", text, re.DOTALL)
        df.at[index, 'Demandante'] = demandante.group(1) if demandante else "No encontrado"
        df.at[index, 'Demandado'] = demandado.group(1) if demandado else "No encontrado"
        df.at[index, 'Expediente_Judicial'] = expediente.group(1) if expediente else "No encontrado"
        df.at[index, 'Juzgado'] = juzgado.group(1).replace('\n', ' ').strip() if juzgado else "No encontrado"
        if "BANCO DE CREDITO DEL PERU" not in demandante.group(1) if demandante else "":
            os.remove(pdf_path)

def main():
    drive.mount('/content/drive')
    df = pd.DataFrame()
    stop_number = 'Remate N° 14062'
    for first_row in range(0, 100, 12):  # Asumiendo que n es 100 por ahora
        response = fetch_page(first_row)
        soup_html, view_state = parse_html(response.content)
        data = extract_data(soup_html)
        temp_df = pd.DataFrame(data)
        df = pd.concat([df, temp_df], ignore_index=True)
        if df[df['Número'] == stop_number].any().any():
            break
    for index, row in df.iterrows():
        response = downloadFile(row['Script ID'])
        process_pdf(response, index, df)
    filtered_df = df[df['Demandante'].str.contains("BANCO DE CREDITO DEL PERU", na=False)]
    columns_to_include = ['Número', 'Demandante', 'Demandado', 'Expediente_Judicial', 'Juzgado']
    df_to_save = filtered_df[columns_to_include]
    df_to_save.to_excel(os.path.join(save_folder, 'Remates_Data.xlsx'), index=False, engine='openpyxl')

if __name__ == '__main__':
    main()


{'Accept': 'application/xml, text/xml, */*; q=0.01', 'Content-Type': 'application/x-www-form-urlencoded', 'Cookie': 'JSESSIONID="tTVzWVs8aA0lJ53sHXPS3qUohGzP8KnpK7n8xaqz.P-Nodo5:SRemaju7-3"'}
{'formBuscarRemateExterno': 'formBuscarRemateExterno', 'formBuscarRemateExterno:listaRemate:0:j_idt171': '', 'javax.faces.ViewState': '1332934851288525934:8043642494852451096'}
<Response [200]>
Failed to download a valid PDF file.
